In [ ]:
from __future__ import print_function, division, absolute_import
import sys
%load_ext autoreload
%autoreload 2

In [ ]:
import cocopod
import cocopod.segment_assignment as sa
import cocopod.topology as t

In [ ]:
#load ply file and convert it to edge notation
edge_topology = t.convert_vface_to_efaces(t.load_vfaces("ply/01_tetrahedron.ply"))
#Explore all topologies
tops = t.explore(edge_topology, True)
#enumarate and name them
tops = t.name_topologies_and_permutations(tops)
#calculate the topological contact order
df = t.calculate_TCO(tops)
df.sort_values(by=["num_AP","num_cross","TCO","stdTCO","segments" ])

In [ ]:
#We chose the the topology with the smallest TCO and stdTCO and only two antiparallel
topology = df.loc['1.10']['segments'] #This is 1.10 or 1.1R
print(topology)
topology = t.standard(topology) 
print(topology)

In [ ]:
#Assign the segments and linkers
gui = sa.segment_assignment_gui(topology)

In [ ]:
#the assignment can be retrived using: 
seg_assignment_str = gui.result_text.value
#When we first designed the tetrahedron there were less segments available. 
#For reproducibility we are using the previous assignment
seg_assignment_str ="""
A->APHshSN:APHshSN
B->P3SN:P4SN
C->BCRSN:BCRSN
D->GCNshSN:GCNshSN
E->P7SN:P8SN
F->P5SN:P6SN
"""
model_name = "TET12SN"
linker = "GSGPG"

seg_assignment = sa.segment_assignments_to_dict(seg_assignment_str)
real_segments = sa.do_assignment_replacements(topology, seg_assignment_str)

#one can also manually edit the sequnce
annotated_seq = sa.get_annotated_sequence(real_segments, 'segments.xlsx', N_tag="M",C_tag="LEHHHHHHHH", linkers=linker)
annotated_seq_edit = sa.sequence_edit_gui(annotated_seq, caption="-".join(real_segments), model_name=model_name)

In [ ]:
#Now write everything to a self-contained config file! 
#Information about pairs and and 3D templates is need and is read from 'segments.xls' sheet 'pairs'.

included_pairs = [":".join(s) for s in seg_assignment.values()]
pairs_info = sa.get_included_pairs_info('segments.xlsx', sheetname='pairs', included_pairs=included_pairs)
sa.write_make_config(model_name, annotated_seq_edit.value, pairs_info, out_name='make_config.py')

# Now run the model building!


The rest of the pipeline is handled via Snakemake (a workflow manager). There is a Snakemake file that can be customized.
Sankemake takes care of dependencies and rerunning tasks and analysis. Make_config.py generates a JSON file with all details (that is less human readable). Next chimera generates a model of the protein. Modeller than bring the various segments together and refines the twist via homology modelling. All the alignment files are generated automatically.


In [ ]:
#this takes a bit less than an hour on a modern intel procesor. 
#One foldin and three homology refinments are run. The homology refinement tasks are run in parallel on 3 threads.

In [ ]:
%%bash

doit -n 3 N_fold=1 N_homology=3

In [ ]:
#view the generated models in chimera
!bash chimera */03-*.pdb

In [ ]:
#or directly in the notebook
import mdtraj 
import mdtraj.html 
from glob import glob
import ipywidgets as widgets
from IPython.display import display

files = glob("*/03-*.pdb")
models = mdtraj.load(files)

def on_value_change(change):
    w.frame=change['new']

slider = widgets.FloatSlider(value=0, min=0, max=models.n_frames-1, step=1, description='Model:')
slider.observe(on_value_change, names='value')
    
mdtraj.html.enable_notebook()


display(slider)
w = mdtraj.html.TrajectoryView(models, secondaryStructure="ribbon")
w

In [ ]:
#Note snakemake workflow managment also supported
!pip install snakemake
!snakemake -j3 --config N_fold=1 N_homology=3